In [40]:
import pandas as pd
import snap

In [41]:
edges = pd.read_csv("da/before_GC/Edges.csv")
nodes = pd.read_csv("da/before_GC/Nodes.csv")

In [42]:
# Create an empty Vector - SNAP data structure
src = snap.TIntV() 
trg = snap.TIntV()
n = snap.TIntV()

# Add elements
[src.Add(element) for element in edges['Source'].tolist()]
[trg.Add(element) for element in edges['Target'].tolist()]
[n.Add(element) for element in nodes['Id'].tolist()]

# Print vector size
print ('Source Size -->',src.Len())
print ('Target Size -->',trg.Len())
print ('Sequence Length -->',n.Len())

Source Size --> 11825009
Target Size --> 11825009
Sequence Length --> 934095


# Graph Creation

In [43]:
# Create a Directed Graph
G1 = snap.TNGraph.New()

# Add Nodes before 
for i in range(0, n.Len()):
    G1.AddNode(n[i])

# adding edges
for i in range(0, src.Len()):
    G1.AddEdge(src[i],trg[i])

# Set node labels
NIdName = snap.TIntStrH() 
for node in n:
    NIdName[node] = str(node)

# Directed network
N1 = snap.TNEANet.New()
print("Directed Graph creation successfuly!")

Directed Graph creation successfuly!


# Giant Component Extraction

## Subset Graph by Removing Low Degree Nodes 

In [44]:
# LIST OF NODES INDEXES TO REMOVE FROM THE DATASET
listA=[]

# DEFINE THE IN-DEGREE AND OUT-DEGREE REMOVAL THRESHOLD
in_deg_remove = 300
out_deg_remove = 300

# GET NODES INDEXES FOR THAT CRITERIA
for NI in G1.Nodes():
    if NI.GetOutDeg()<out_deg_remove and NI.GetInDeg()<in_deg_remove:
        listA.append(NI.GetId())

# CREATE SNAP VECTOR CONSIST THESE LIST
V = snap.TIntV()
[V.Add(element) for element in listA]

# Nodes & Edges - BEFORE
print("Before:")
print("========")
print('Edges = ',G1.GetEdges())
print('Nodes = ',G1.GetNodes())

# DELETE NODES AND EDGES FROM GRAPH (NOT FROM THE DATASET)
snap.DelNodes(G1, V)

# Nodes & Edges - AFTER
print("After:")
print("========")
print('Edges = ',G1.GetEdges())
print('Nodes = ',G1.GetNodes())

Before:
Edges =  11825009
Nodes =  934095
After:
Edges =  589286
Nodes =  4477


# Subset Data According to Nodes & Edges Removal from the Graph

In [45]:
# REMOVE LOW-DEGREE NODES
nodes = nodes[~nodes.index.isin(listA)]

# REMOVE SOURCE LOW-DEGREE NODES FROM THE EDGES
edges = edges[~edges.Target.isin(listA)]

# REMOVE TARGET LOW-DEGREE NODES FROM THE EDGES
edges = edges[~edges.Source.isin(listA)]

# Data extraction to csv

In [46]:
# EXTRACT NODES
nodes.to_csv("da/after_GC/Nodes_hard"+str(in_deg_remove)+".csv", index = False)

# EXTRACT EDGES 
edges.to_csv("da/after_GC/Edges_hard"+str(in_deg_remove)+".csv", index = False)